In [1]:
import pandas as pd
import random
from pathlib import Path
import numpy as np
import psycopg2
from datetime import date, timedelta

# Данные

## Подготовка данных для таблицы book

In [2]:
books_num = 10000
book_author_num = 7000
book_lang_num = 15
book_publisher_num = 3000

In [3]:
directory = Path("input/book")
book = pd.concat([pd.read_csv(f).drop(
    ['RatingDist1', 'RatingDist4', 'RatingDistTotal', 'RatingDist2', 'RatingDist5', 'RatingDist3', 'Id'], axis=1) for f
    in directory.glob("*.csv")], ignore_index=True)

# book = book.drop(['pagesNumber', 'Count of text reviews'], axis=1)
book.Language = book.Language.fillna('eng')
book.pagesNumber = book.pagesNumber.fillna(random.randint(250, 1500))
book.pagesNumber = book.pagesNumber.astype('int')
book.Publisher = book.Publisher.fillna('Alpina')
book.Description = book.Description.fillna('No description')
book['Genre'] = np.random.choice(
    ['Action and Adventure', 'Classics', 'Comic Book', 'Detective', 'Mystery', 'Fantasy', 'Historical Fiction',
     'Literary Fiction', 'Horror', 'Romance', 'Science Fiction', 'Short Stories', 'Suspense', 'Thrillers',
     'Biographies', 'Autobiographies', 'Cookbooks', 'Essays', 'History', 'Memoir', 'Poetry', 'Self-Help', 'True Crime'],
    len(book))
book.ISBN = book.ISBN.fillna(random.randint(1000, 10000000))
book = book.drop(["ISBN"], axis=1)
book = book.drop(["Count of text reviews"], axis=1)

book.loc[book['PublishYear'] > 2023, 'PublishYear'] = random.randint(1900, 2023)
book.loc[book['PublishMonth'] > 12, 'PublishMonth'] = random.randint(1, 12)
book.loc[book['PublishDay'] > 31, 'PublishDay'] = random.randint(1, 31)
book.loc[(book['Language'] == "en-US") | (book['Language'] == "en-GB") | (book['Language'] == "en-CA") | (
        book['Language'] == "--"), 'Language'] = "eng"

book = book[:books_num]
book_copy = book.copy()
book_copy = book_copy.drop(columns=['Authors', 'Language', 'Genre'])
book_copy['Publisher'] = [random.randint(1, 25000) for i in range(books_num)]
book_copy.columns = ['book_title',
                     'book_rating',
                     'book_publish_year',
                     'book_publish_month',
                     'book_publish_day',
                     'book_publisher',
                     'book_num_reviews',
                     'book_page_num',
                     'book_desc']

book_copy.to_csv("output/book.csv", index=False)
book_copy

,book_title,book_rating,book_publish_year,book_publish_month,book_publish_day,book_publisher,book_num_reviews,book_page_num,book_desc
0,A Passion to Preserve: Gay Men as Keepers of C...,3.75,2005,1,9,7970,6,298,"From large cities to rural communities, gay me..."
1,Culture Keepers-Florida: Oral History of the A...,0.00,2006,9,7,395,0,140,No description
2,Holiday Favorites: The Best of the Williams-So...,4.55,2004,1,4,7132,2,336,This collector's edition features fabulous ful...
3,"Soups, Salads & Starters: the Best of Williams...",4.70,2004,1,4,5004,2,336,No description
4,Breakfasts & Brunches,3.88,1997,1,4,13526,0,108,"America's most respected cookware retailer, th..."
...,...,...,...,...,...,...,...,...,...
9995,Starting Riding,4.00,1992,9,4,13297,0,32,"A guide for new riders, covering everything fr..."
9996,The Perilous Frontier: Nomadic Empires and China,4.33,1992,9,7,2381,9,325,"Around 800 BC, the Eurasian steppe underwent a..."
9997,Postmodernism: The Twilight of the Real,3.67,1990,9,5,7899,0,160,No
9998,Perilous Passage (pb): A Narrative of the Mont...,4.00,1995,1,1,18990,0,214,In 1862 Edwin Ruthven Purple seized the chance...


In [4]:
book

,Name,Authors,Rating,PublishYear,PublishMonth,PublishDay,Publisher,CountsOfReview,Language,pagesNumber,Description,Genre
0,A Passion to Preserve: Gay Men as Keepers of C...,Will Fellows,3.75,2005,1,9,University of Wisconsin Press,6,eng,298,"From large cities to rural communities, gay me...",Horror
1,Culture Keepers-Florida: Oral History of the A...,Deborah Johnson-Simon,0.00,2006,9,7,Authorhouse,0,eng,140,No description,Poetry
2,Holiday Favorites: The Best of the Williams-So...,Allen Rosenberg,4.55,2004,1,4,Oxmoor House,2,eng,336,This collector's edition features fabulous ful...,Poetry
3,"Soups, Salads & Starters: the Best of Williams...",Allan Rosenberg,4.70,2004,1,4,Oxmoor House,2,eng,336,No description,Self-Help
4,Breakfasts & Brunches,Time-Life Books,3.88,1997,1,4,Time Life Medical,0,eng,108,"America's most respected cookware retailer, th...",Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Starting Riding,Helen Edom,4.00,1992,9,4,Educational Development Corporation,0,eng,32,"A guide for new riders, covering everything fr...",Action and Adventure
9996,The Perilous Frontier: Nomadic Empires and China,Thomas J. Barfield,4.33,1992,9,7,Wiley-Blackwell,9,eng,325,"Around 800 BC, the Eurasian steppe underwent a...",Detective
9997,Postmodernism: The Twilight of the Real,Neville Wakefield,3.67,1990,9,5,Pluto Press,0,eng,160,No,Short Stories
9998,Perilous Passage (pb): A Narrative of the Mont...,Edwin Purple,4.00,1995,1,1,Montana Historical Society Press,0,eng,214,In 1862 Edwin Ruthven Purple seized the chance...,Fantasy


## Данные для таблиц
- book
- book_genre
- book_lang
- book_publisher
- book_author
- people

In [5]:
people = pd.read_csv("output/people.csv")
people

,people_name,people_surname,people_patronymic,people_sex,people_address,people_tel_num,people_pass_num,people_email,people_password,people_role
0,Смирнова,Василиса,Ильинична,False,17085,89283672851,221174457,ayHKyWw@gmail.com,sha256$MqcBjLztOvPXEQlc$77ed258f3dbff6c73172c2...,1
1,Агеева,Софья,Богдановна,False,13761,89572177451,671491856,yQyIadw@gmail.com,sha256$HF6takUpWYXoXrsO$608cfb5a675aee9ec0c634...,1
2,Шаров,Серафим,Матвеевич,True,23831,89577178345,145915573,jyVQLnA@gmail.com,sha256$19w99EAYQ1cayvl2$ca0ceab4baf7c7eeb32d01...,1
3,Кузьмин,Сергей,Александрович,True,4185,89025582245,229938897,kAsXLXU@gmail.com,sha256$o94TXMvvEP2Ar7sV$28784c6cff28dcf09de611...,1
4,Дмитриев,Дмитрий,Матвеевич,True,18342,89695282505,173322769,mKkVcXX@gmail.com,sha256$N0AQUIOlWW4E2ii3$f55de07d78126a906d8be5...,1
...,...,...,...,...,...,...,...,...,...,...
595,Родионов,Михаил,Кириллович,True,12850,89270717235,838082164,RgBhuyo@gmail.com,sha256$GP4LAlcWvoXZG4cr$2bfda1ac6883ae20de237c...,1
596,Иванова,Таисия,Константиновна,False,38541,89380030963,651566879,jAbTDxi@gmail.com,sha256$hzvVGne97YHiYMG1$f45c5d60ecacd271a25c72...,1
597,Захарова,Вероника,Никитична,False,38082,89124451789,290729844,heKUMkZ@gmail.com,sha256$JtHP2UBVZ3ZJfIYI$7db7126adf9871d3c98a74...,1
598,Алексеева,Софья,Александровна,False,9432,89173019662,478551527,MpzFpnV@gmail.com,sha256$jj3aiDpZUR2xdfmv$04cca8dc3a3d8e3fd13d90...,1


In [6]:
book_author = pd.DataFrame(data=book.Authors.unique(), columns=['author_name'])
author_sex = [random.choice([True, False]) for i in range(book_author.count()[0])]
book_author['author_sex'] = author_sex
book_author = book_author[:book_author_num]
book_author.to_csv("output/book_author.csv", index=False)
book_author

,author_name,author_sex
0,Will Fellows,False
1,Deborah Johnson-Simon,True
2,Allen Rosenberg,True
3,Allan Rosenberg,False
4,Time-Life Books,True
...,...,...
6995,Hermano Vianna,True
6996,Gilbert Morris,False
6997,Ernest B. Furgurson,True
6998,Gregory Jaynes,True


In [7]:
book_book_author = pd.DataFrame(columns=[['book_book_id', 'book_author_author_id']])
book_book_id = [random.randint(1, books_num) for i in range(books_num * 2)]
book_author_author_id = [random.randint(1, book_author_num) for i in range(books_num * 2)]
book_book_author['book_book_id'] = book_book_id
book_book_author['book_author_author_id'] = book_author_author_id
book_book_author.to_csv("output/book_book_author.csv", index=False)
book_book_author

,book_book_id,book_author_author_id
0,8989,2207
1,6822,4291
2,6395,4228
3,6546,1564
4,7382,4661
...,...,...
19995,4255,5688
19996,9638,4748
19997,7940,2015
19998,2488,6281


In [8]:
book_genre = pd.DataFrame(data=book.Genre.unique(), columns=['genre_title'])
book_genre.to_csv("output/book_genre.csv", index=False)
book_genre

,genre_title
0,Horror
1,Poetry
2,Self-Help
3,Fantasy
4,Science Fiction
5,Mystery
6,Essays
7,Biographies
8,Literary Fiction
9,Suspense


In [9]:
book_book_genre = pd.DataFrame(columns=[['book_book_id', 'book_genre_genre_id']])
book_book_id = [random.randint(1, books_num) for i in range(books_num * 2)]
book_genre_genre_id = [random.randint(1, 23) for i in range(books_num * 2)]
book_book_genre['book_book_id'] = book_book_id
book_book_genre['book_genre_genre_id'] = book_genre_genre_id
book_book_genre.to_csv("output/book_book_genre.csv", index=False)
book_book_genre

,book_book_id,book_genre_genre_id
0,7074,3
1,2730,7
2,7407,21
3,2859,2
4,8344,19
...,...,...
19995,7768,11
19996,4675,5
19997,4850,9
19998,8753,18


In [10]:
book_lang = pd.DataFrame(data=book.Language.unique(), columns=['lang_title'])
book_lang.to_csv("output/book_lang.csv", index=False)
book_lang

,lang_title
0,eng
1,ita
2,spa
3,fre
4,eus
5,ger
6,jpn
7,mul
8,per
9,tha


In [11]:
book_book_lang = pd.DataFrame(columns=[['book_book_id', 'book_lang_lang_id']])
book_book_id = [random.randint(1, books_num) for i in range(books_num * 2)]
book_lang_lang_id = [random.randint(1, book_lang_num) for i in range(books_num * 2)]
book_book_lang['book_book_id'] = book_book_id
book_book_lang['book_lang_lang_id'] = book_lang_lang_id
book_book_lang.to_csv("output/book_book_lang.csv", index=False)
book_book_lang

,book_book_id,book_lang_lang_id
0,6925,15
1,1604,15
2,5825,12
3,4730,6
4,1693,9
...,...,...
19995,9480,9
19996,3834,13
19997,1864,8
19998,8020,5


In [12]:
book_publisher = pd.DataFrame(data=book.Publisher.unique(), columns=['publisher_name'])
publisher_desc = [f"Desc of {book_publisher.iloc[i, 0]}" for i in range(book_publisher.count()[0])]
book_publisher['publisher_desc'] = publisher_desc
book_publisher = book_publisher[:book_publisher_num]
book_publisher.to_csv("output/book_publisher.csv", index=False)
book_publisher

,publisher_name,publisher_desc
0,University of Wisconsin Press,Desc of University of Wisconsin Press
1,Authorhouse,Desc of Authorhouse
2,Oxmoor House,Desc of Oxmoor House
3,Time Life Medical,Desc of Time Life Medical
4,Pocket Books,Desc of Pocket Books
...,...,...
2995,Bobbs-Merrill,Desc of Bobbs-Merrill
2996,Acanthus Press,Desc of Acanthus Press
2997,American Geophysical Union,Desc of American Geophysical Union
2998,Editions Heimdal,Desc of Editions Heimdal


In [13]:
address = pd.read_csv("input/address/open_pubs.csv").drop(["fas_id", "name", "local_authority", "easting", "northing"],
                                                          axis=1)
address.columns = ['address_title', 'address_postcode', 'address_latitude', 'address_longitude']
address = address[:10000]
address.to_csv("output/address.csv", index=False)
address

,address_title,address_postcode,address_latitude,address_longitude
0,"Upper Street, Stratford St Mary, COLCHESTER, E...",CO7 6LW,51.97039,0.979328
1,"Egremont Street, Glemsford, SUDBURY, Suffolk",CO10 7SA,52.094427,0.668408
2,"7 Market Hill, SUDBURY, Suffolk",CO10 2EA,52.038683,0.730226
3,"Lower Street, Stratford St Mary, COLCHESTER, E...",CO7 6JS,51.966211,0.972091
4,"Lion Road, Glemsford, SUDBURY, Suffolk",CO10 7RF,52.102815,0.666893
...,...,...,...,...
9995,"2-3 Creed Lane, London",EC4V 5BR,51.513429,-0.101202
9996,"Retail Unit, 8 Devonshire Row, London",EC2M 4RH,51.516935,-0.080142
9997,"60 Holborn Viaduct, London",EC1A 2FD,51.516994,-0.103662
9998,"1 Bow Churchyard, London",EC4M 9DQ,51.513493,-0.094065
